## Welcome to the local demo notebook!

### Start by installing the dependencies

In [ ]:
%cd ..
%pip install -r requirements.txt
%cd demo

### Import necessary modules

In [ ]:
import os
import sys
import torch
sys.path.append('..')
from src import defdevice

# If you have a cuda device, specify in the force_device function
if torch.cuda.is_available():
    defdevice.force_device('cuda:0')
else:
    defdevice.force_cpu('cpu')

from src import nns
from src import embedders
from src import segmodel
from demo.demo_functions import run_image, run_video

#### Initialize the model and the embedders

In [ ]:
image_embr = embedders.M2FImageEmbedder()
text_embr = embedders.CLIPTextEmbedder()
model = nns.Linear(1, 1)

### Load the model and test it out

In [ ]:
model.load("../weights/CSRUGDFineTuned") # Specify the weights you want to load
smodel = segmodel.CSModel(image_embr, text_embr, model)

file_list = ['../FP/fp01.jpeg'] # Images to run the model on.
labels = ['void,sky,sign',
          'concrete,asphalt',
          'gravel,grass,dirt,sand,mulch',
          'rock,rock-bed',
          'water',
          'tree,pole,vehicle,container/generic-object,building,log,bicycle,person,fence,bush,picnic-table,bridge'] # Labels to look for.

labels = [st.split(',') for st in labels]

rough_labels = ['background', 'smooth terrain', 'rough terrain', 'bumpy terrain', 'forbidden terrain', 'obstacle']

for path in file_list:
    run_image(smodel, path, labels, multilabel = True, aggregation = 'max', rough_labels = rough_labels)

### Test without the rough labels

In [ ]:
model.load("../weights/CSRUGDFineTuned") # Specify the weights you want to load
smodel = segmodel.CSModel(image_embr, text_embr, model)

file_list = ['../FP/fp01.jpeg'] #Images to run the model on.
labels = ['dirt road', 'tree', 'sky', 'fence' 'other'] # Labels to look for.

for path in file_list:
    run_image(smodel, path, labels)

### Test a different model

In [ ]:
model.load("../weights/CSTableModel") # Specify the weights you want to load
smodel = segmodel.CSModel(image_embr, text_embr, model)

file_list = ['../FP/fp01.jpeg'] # Images to run the model on.
labels = ['dirt road, path, ground', 'moss, grass', 'trees', 'bike', 'helmet', 'other'] # Labels to look for.

for path in file_list:
    run_image(smodel, path, labels)

### Test out video

In [ ]:
model.load("../weights/CSTableModel") # Specify the weights you want to load
smodel = segmodel.CSModel(image_embr, text_embr, model)

labels = [ 'Mountain', 'Grass', 'Moss', 'Rock', 'Other'] # Labels to look for.

path = '../FP/Vid1.mp4' # Video path.
width, height = (468, 256)
output_path = '../FP/generated_video.mp4'
fps = 30
frame_count = 50000 # Maximum amount of frames to segment.
frame_average = 5 # How many frames to average segmentation over.
draw_period = 300 # How frequently to output example segmentations.

run_video(path, smodel, labels, width, height, output_path, fps, frame_count, frame_average, draw_period, web = False)